In [1]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

template = "You are a helpful assistant that translates english to pirate."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
example_human = HumanMessagePromptTemplate.from_template("Hi")
example_ai = AIMessagePromptTemplate.from_template('Argh me mateys')
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, example_human, example_ai, human_message_prompt])
chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run("I love programming")

c:\Users\veace\AppData\Local\Programs\Python\Python311\Lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.1) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


"I be lovin' the art of code plunderin'"

In [4]:
from langchain import PromptTemplate, FewShotPromptTemplate

examples = [
    {"query": "What's the weather like?",
     "answer": "It's raining cats and dogs, better bring and umbrella!"},
    {"query": "How old are you?",
     "answer": "Age is just a number, but I'm timeless."}
]

example_template = """
User: {query}
Answer: {answer}
"""

example_prompt = PromptTemplate(input_variables=['query', 'answer'], template=example_template)

prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""

suffix = """
User: {query}
AI: 
"""

few_shot_prompt_template = FewShotPromptTemplate(examples=examples,
                                                 example_prompt=example_prompt,
                                                 prefix=prefix,
                                                 suffix=suffix,
                                                 input_variables=['query'],
                                                 example_separator="\n\n")

chain = LLMChain(llm=chat, prompt=few_shot_prompt_template)
chain.run("What's the secret to happiness?")

'A lifetime supply of chocolate and a good sense of humor!'

#### Example selector

In [9]:
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "energetic", "antonym": "lethargic"},
    {"word": "sunny", "antonym": "gloomy"},
    {"word": "windy", "antonym": "calm"},
]

example_template = """
Word: {word}
Antonym: {antonym}
"""

example_prompt = PromptTemplate(input_variables=['word', 'antonym'], template=example_template)

example_selector = LengthBasedExampleSelector(examples=examples, example_prompt=example_prompt, max_length=25)

dynamic_prompt = FewShotPromptTemplate(example_selector=example_selector,
                                       example_prompt=example_prompt,
                                       prefix="Give the antonym of every input",
                                       suffix="Word: {input}\nAntonym:",
                                       input_variables=['input'],
                                       example_separator="\n\n")

chain = LLMChain(llm=chat, prompt=dynamic_prompt)
chain.run("big good")

'small bad'

Semantic similarity example selector

In [11]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import DeepLake
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(input_variables=['input', 'output'],
                                template="Input: {input}\nOutput: {output}")

examples = [
    {"input": "0°C", "output": "32°F"},
    {"input": "10°C", "output": "50°F"},
    {"input": "20°C", "output": "68°F"},
    {"input": "30°C", "output": "86°F"},
    {"input": "40°C", "output": "104°F"},
]

my_activeloop_org_id = 'veaceslavcalestru'
my_activeloop_dataset_name = "langchain_course_fewshot_selector"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path)

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

example_selector = SemanticSimilarityExampleSelector.from_examples(examples, embeddings, db, k=1)

similar_prompt = FewShotPromptTemplate(example_selector=example_selector,
                                       example_prompt=example_prompt,
                                       prefix="Convert the temperature from Celsius to Fahrenheit",
                                       suffix="Input: {temperature}\nOutput:",
                                       input_variables=['temperature'])

print(similar_prompt.format(temperature="10°C"))
print(similar_prompt.format(temperature="30°C"))

similar_prompt.example_selector.add_example({"input": "50°C", "output": "122°F"})
print(similar_prompt.format(temperature="40°C"))

Deep Lake Dataset in hub://veaceslavcalestru/langchain_course_fewshot_selector already exists, loading from the storage
Deep Lake Dataset in ./deeplake/ already exists, loading from the storage


Dataset(path='./deeplake/', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (10, 1536)  float32   None   
    id        text      (10, 1)      str     None   
 metadata     json      (10, 1)      str     None   
   text       text      (10, 1)      str     None   
Convert the temperature from Celsius to Fahrenheit

Input: 10°C
Output: 50°F

Input: 10°C
Output:
Convert the temperature from Celsius to Fahrenheit

Input: 30°C
Output: 86°F

Input: 30°C
Output:


Dataset(path='./deeplake/', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (11, 1536)  float32   None   
    id        text      (11, 1)      str     None   
 metadata     json      (11, 1)      str     None   
   text       text      (11, 1)      str     None   
Convert the temperature from Celsius to Fahrenheit

Input: 40°C
Output: 104°F

Input: 40°C
Output:
